## Julia on Colaboratory ##

[Colaboratory](https://colab.research.google.com) does not provide native support for the [Julia programming language](https://julialang.org). However, since Colaboratory gives you root access to the machine that runs your notebook (the *“runtime”* in Colaboratory terminology), we can install Julia support by uploading a specially crafted Julia notebook  – *this* notebook. We then install Julia and [IJulia](https://github.com/JuliaLang/IJulia.jl) ([Jupyter](https://jupyter.org)/Colaboratory notebook support) and reload the notebook so that Colaboratory detects and initiates what we installed.

In brief:

1. **Run the cell below**
2. **Reload the page**
3. **Edit the notebook name and start hacking Julia code below**

**If your runtime resets**, either manually or if left idle for some time, **repeat steps 1 and 2**.

### Acknowledgements ###

This hack by Pontus Stenetorp is an adaptation of [James Bradbury’s original Colaboratory Julia hack](https://discourse.julialang.org/t/julia-on-google-colab-free-gpu-accelerated-shareable-notebooks/15319/27), that broke some time in September 2019 as Colaboratory increased their level of notebook runtime isolation. There also appears to be CUDA compilation support installed by default for each notebook runtime type in October 2019, which shaves off a good 15 minutes or so from the original hack’s installation time.

In [0]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add Plots; add PyPlot; add IJulia; add Knet;"'
julia -e 'using Pkg; pkg"build Knet;"'

--2020-03-19 19:18:05--  https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz [following]
--2020-03-19 19:18:05--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.3.48, 2607:f8b0:4026:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.3.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95929584 (91M) [application/x-gzip]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[===================>]  

In [1]:
using Knet
# Test if Knet is using gpu
Knet.gpu()

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1273
┌ Warning: You are using CUDNN 7.6.5 for CUDA 10.1.0 with CUDA toolkit 10.0.130; these might be incompatible.
└ @ CuArrays /root/.julia/packages/CuArrays/A6GUx/src/CuArrays.jl:128


0

In [2]:
# Setup display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in
 ("Knet","IterTools","Plots"); haskey(Pkg.installed(),p) || Pkg.add(p); end

using Base.Iterators: flatten
using IterTools: ncycle, takenth
using Statistics: mean
using Knet: Knet, conv4, pool, mat, KnetArray, nll, zeroone, progress, sgd, param, param0, dropout, relu, Data, gpu
import Pkg
IJulia.set_max_stdio(1<<25)

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
  [c8e1da08] + IterTools v1.3.0
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


┌ Info: Precompiling IterTools [c8e1da08-722c-5040-9ed9-7db0dc04731e]
└ @ Base loading.jl:1273


In [0]:
# Load data (see mnist.ipynb)
include(Knet.dir("data","mnist.jl"))  # Load data
dtrn,dtst = mnistdata(batchsize=50);   

In [204]:
# For running experiments
function trainresultsadam(model; o...)
        r = ((1)  for x in takenth(progress(adam(model,ncycle(dtrn,10))),length(dtrn)))
        r = reshape(collect(Float32,flatten(r)),(1,:))
        Knet.gc() # To save gpu memory
end

trainresultsadam (generic function with 2 methods)

In [205]:
# Squash function definition
function squash(s; axis=2, eps=1e-7)
  norm2 = sum(abs2.(s),dims=axis);
  scaled = s./sqrt.(norm2);
  result = (norm2./(norm2.+1)).*scaled;
end

squash (generic function with 1 method)

In [206]:
# Convolutional Layer Definition
struct Conv; w; b; f; p; s; end
(c::Conv)(x) = c.f.((conv4(c.w, dropout(x,c.p),padding=0,stride=c.s) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0,s=1) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop,s)

Conv

In [207]:
# Capsule Network Definition
struct Capsule; w;w1;w2;cx;bs;end
(caps::Capsule)(x) = (
x = reshape(x,(caps.w1,caps.w2,size(caps.w,2),caps.cx,caps.bs));
x = permutedims(x,(1,2,4,3,5));
x = reshape(x,(size(caps.w,1),size(caps.w,2),1,1,caps.bs));

a = convert(KnetArray{Float32},ones(size(caps.w,1),size(caps.w,2),size(caps.w,3),size(caps.w,4),caps.bs));

w_expand = a.*caps.w;

u_hat = sum(w_expand.*x,dims=2);
u_hat = reshape(u_hat,(1152,10,16,caps.bs));

b = convert(KnetArray{Float32},zeros(size(caps.w,1),size(caps.w,3),1,caps.bs));

for i = 1 : 3
    c = softmax(b,dims=2);
    s = permutedims(reshape(sum((c.*u_hat),dims=1),16,10,caps.bs),(2,1,3));
    v = squash(s);
    v = reshape(v,(1,10,16,caps.bs));
    A = u_hat.*v;
    aggrement = sum(A,dims = 3);
    b = b.+aggrement;
end;

v= reshape(v,(10,16,caps.bs));
probabilities = sum(v,dims=2);

result = reshape(probabilities,10,caps.bs);
)

Capsule(w1::Int,w2::Int,cx::Int,lenx::Int,cy::Int,leny::Int,bs::Int) = Capsule(param(w1*w2*cx,lenx,cy,leny),w1,w2,cx,bs)

Capsule

In [208]:
# Dense Layers Definition
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [0]:
# Chain of layers definition
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [210]:
(x,y) = first(dtrn);
summary(x)

"28×28×1×50 KnetArray{Float32,4}"

In [211]:
#Defining baseline layer
baseline_capsnet =   Chain(Conv(9,9,1,256,), 
                 Conv(9,9,256,256,s = 2),
                Capsule(6,6,32,8,10,16,50),)
println(summary.(l.w for l in baseline_capsnet.layers));


["9×9×1×256 Param{KnetArray{Float32,4}}", "9×9×256×256 Param{KnetArray{Float32,4}}", "1152×8×10×16 Param{KnetArray{Float32,4}}"]


In [212]:
baseline_trained_capsnet = Knet.load("capsnet_trained.jld2","baseline_capsnet_trnd")

Chain((Conv(P(KnetArray{Float32,4}(9,9,1,256)), P(KnetArray{Float32,4}(1,1,256,1)), NNlib.relu, 0, 1), Conv(P(KnetArray{Float32,4}(9,9,256,256)), P(KnetArray{Float32,4}(1,1,256,1)), NNlib.relu, 0, 2), Capsule(P(KnetArray{Float32,4}(1152,8,10,16)), 6, 6, 32, 50)))

In [213]:
baseline_trained_capsnet(x)


10×50 KnetArray{Float32,2}:
 -3.53138  -3.79172  -3.53819  …  -3.59066  -2.80799  -3.49524
 -3.73842  -3.7127   -3.75571     -3.65101  -3.33078  -3.84533
 -3.11653  -3.79185  -3.59811     -3.7248   -3.1626    3.94741
 -3.70236  -3.76109   3.92701     -3.78277  -3.40408  -3.64598
  3.60998  -3.6654   -3.70642      3.92579  -2.75019  -3.44083
 -3.59037  -3.79953  -3.60963  …  -3.5869   -3.7349   -3.52778
 -3.71159  -3.72637  -3.65872     -3.85041  -3.36216  -3.82185
 -3.41429  -3.82285  -3.64325     -3.64111  -3.62061  -3.73497
 -3.73651  -3.6986   -3.49845     -3.62178   2.74298  -3.7451 
 -3.64936   3.97108  -3.77033     -3.60173  -3.56855  -3.75376

In [217]:
zeroone(baseline_trained_capsnet,dtst)

0.007600000000000051

In [218]:
zeroone(baseline_trained_capsnet,dtrn)

0.0020000000000000018

In [0]:
# Training can be done with uncommenting the function below
# trainresultsadam(baseline_capsnet);

In [221]:
#After training, trained network can be saved
file = "capsnet_trained.jld2"
Knet.save(file,"baseline_capsnet_trnd",baseline_capsnet)

Error encountered while saving "/content/capsnet_trained.jld2".

Fatal error:


InterruptException: ignored